In [1]:
import cv2
import os
import mediapipe as mp
import numpy as np
import random
import pandas as pd
import pickle
from utilities import *
from joint_angles import JointAngle, Angles
from body_parts import BodyPart

In [2]:
n = 8
angles = [0] * n
bp = BodyPart()

In [26]:
mpPose = mp.solutions.pose
video = '../assets/sample-video/sample-video05.mp4'
cap = cv2.VideoCapture(video)

In [27]:
data = {
    'Angles': [],
    'Phase': [],
    'Score': []
}

In [28]:
output_folder = '../assets/sample-frames/sample-video05/'
os.makedirs(output_folder, exist_ok=True)
frameCnt = 0

with mpPose.Pose(min_detection_confidence=0.75, min_tracking_confidence=0.75) as pose:
    while cap.isOpened():
        ret, image = cap.read()

        frameName = f'frame_{frameCnt}.jpg'
        framePath = os.path.join(output_folder,frameName)
        cv2.imwrite(framePath, image)
        frameCnt += 1

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image.flags.writeable = True
        
        try:
            lm = results.pose_landmarks.landmark
            jointAngle = JointAngle(lm)
            bp.cords = detect_joints(lm)
            angles = jointAngle.body_angles(bp)
            data['Angles'].append(angles)
            data['Phase'].append(None)
            data['Score'].append(None)
        except:
            pass

error: OpenCV(4.8.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


In [29]:
data

{'Angles': [[9, 179, 180, 0, 180, 11, 175, 171],
  [8, 179, 180, 0, 180, 7, 177, 180],
  [8, 179, 179, 0, 180, 7, 179, 179],
  [8, 179, 179, 0, 180, 7, 179, 179],
  [8, 179, 179, 0, 180, 8, 179, 178],
  [8, 179, 179, 0, 180, 8, 178, 177],
  [7, 179, 179, 0, 180, 8, 178, 177],
  [7, 180, 180, 0, 179, 9, 178, 178],
  [6, 179, 179, 0, 180, 9, 178, 178],
  [4, 179, 179, 0, 180, 10, 178, 178],
  [3, 179, 179, 0, 180, 10, 178, 178],
  [4, 179, 178, 0, 179, 10, 178, 178],
  [5, 180, 177, 0, 179, 9, 177, 179],
  [5, 179, 176, 0, 178, 9, 176, 179],
  [5, 179, 176, 0, 178, 11, 176, 176],
  [6, 179, 176, 0, 179, 9, 176, 176],
  [7, 179, 175, 0, 179, 8, 177, 176],
  [9, 178, 172, 0, 179, 9, 176, 174],
  [10, 176, 171, 0, 179, 10, 176, 174],
  [11, 175, 168, 0, 180, 16, 60, 162],
  [11, 176, 165, 0, 180, 14, 160, 164],
  [10, 176, 161, 0, 180, 12, 162, 167],
  [12, 176, 129, 0, 180, 12, 155, 168],
  [15, 179, 123, 0, 180, 11, 164, 169],
  [16, 173, 88, 0, 180, 7, 177, 176],
  [17, 167, 80, 0, 179, 

In [25]:
df = pd.DataFrame(data)
df
df.to_csv('../dataset/angles05.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
features = df['Angles'].to_list()
labels = label_encoder.fit_transform(df['Phase'])

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(Xtrain, Ytrain)
Ypred = clf.predict(Xtest)

In [ ]:
filename = '../model/PhaseClassifier.pickle'
pickle.dump(clf, open(filename, "wb"))

In [ ]:
scores = {
    'Neck': [],
    'Left Arm': [],
    'Right Arm': [],
    'Back': [],
    'Abdomen': [],
    'Internal': [],
    'Left Leg': [],
    'Right Leg': [],
    'Angle': [],
}

In [ ]:
df = pd.DataFrame(scores)
df

In [ ]:
for i, angle in enumerate(scores):
    print(i, angle)

In [ ]:
degrees = 180

for degree in range(degrees + 1):
    score = 0
    scores['Angle'].append(degree)
    
    for i, angle in enumerate(scores):
        if i == 8:
            continue
        delta = abs(degree - angles[i])
        score = round((1 - delta / degrees) * 100)
        scores[str(angle)].append(score)    

In [ ]:
df = pd.DataFrame(scores)
df

In [ ]:
selectedAngles = df.columns.difference(['Angle'])
selectedAngles

In [ ]:
data = {
    'Angles': [],
    'Score': []
}
for i in range(0, 1000):
    score = 0
    angles = []

    for angle in selectedAngles:
        degree = random.randint(0, 180)
        score += df[angle][degree]
        angles.append(np.array(degree))
    data['Angles'].append(np.array(angles))
    data['Score'].append(np.array(round(score / 8)))

data = pd.DataFrame(data)
# data        

In [ ]:
print(data['Score'].max())
print(data['Score'].min())

In [ ]:
data.to_csv('scores.csv')

In [ ]:
for i in range(0, 1000):
    if abs(data['Angles'][i][3] - data['Angles'][i][6]) > 5:
        data['Angles'][i][3] = data['Angles'][i][6] + random.randint(-4, 4)

In [ ]:
data

In [ ]:
for i in range(len(data)):
    if data['Angles'][i][3] < 90 or data['Angles'][i][6] < 90:
        data['Score'][i] = 1
    else:
        data['Score'][i] = 0

In [ ]:
data

In [ ]:
data = {
    'Angles': [],
    'Score': []
}

angles = []
for angle in selectedAngles:
    if 1 < df[angle].min():
        index = df[df[angle] == df[angle].min()]['Angle'].iloc[0]
    elif 1 > df[angle].max():
        index = df[df[angle] == df[angle].max()]['Angle'].iloc[0]
    else:
        index = df[df[angle] == 1]['Angle'].iloc[0]
    print(index)


In [ ]:
data = {
    'Angles': [],
    'Score': []
}

for score in range(0, 101):
    angles = []
    for angle in selectedAngles:
        if score < df[angle].min():
            index = df[df[angle] == df[angle].min()]['Angle'].iloc[0]
        elif score > df[angle].max():
            index = df[df[angle] == df[angle].max()]['Angle'].iloc[0]
        else:
            index = df[df[angle] == score]['Angle'].iloc[0]
        angles.append(np.array(index))
    angles = np.array(angles)

    data['Angles'].append(angles)
    data['Score'].append(score)

data = pd.DataFrame(data)
data
        

In [ ]:
features = np.array(data['Angles'].to_list())
labels = np.array(data['Score'])

In [ ]:
labels

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from statsmodels.tools.eval_measures import mse

In [ ]:
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(features)

In [ ]:
data['Cluster'] = kmeans.labels_

In [ ]:
data

In [ ]:
features = data['Angles'].to_list()
labels = data['Score'].to_list()

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=1)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(Xtrain, Ytrain)
Ypred = clf.predict(Xtest)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
Ypred

In [ ]:
np.array(Ytest)

In [ ]:
svr = svm.SVR(kernel='linear')
svr.fit(Xtrain, Ytrain)

In [ ]:
Ypred = svr.predict(Xtest)

In [ ]:
Ypred

In [ ]:
Ytest

In [ ]:
filename = 'scores.csv'

In [ ]:
features = df['Angle']
labels = df['Left Arm']
features = np.array(features)
labels = np.array(labels)

In [ ]:
features = features.reshape(-1, 1)

In [ ]:
features.shape

In [ ]:
features

In [ ]:
labels.shape

In [ ]:
labels

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn import svm
from statsmodels.tools.eval_measures import mse

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=1)

In [ ]:
model = LinearRegression()
reg = model.fit(Xtrain, Ytrain)

In [ ]:
print(reg.score(Xtrain, Ytrain))

In [ ]:
Ypred = reg.predict(Xtest)

In [ ]:
Ypred

In [ ]:
Ytest